In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [8]:
df_all = pd.read_csv('fanfic_regression_data_curated.tsv', sep = '\t')

In [9]:
len(df_all)

701599

In [10]:
df_all['ChapterIndex'].fillna(0, inplace=True)

In [11]:
df_all.head(2)

,Bookmarks,ChapterIndex,Chapters,Comments,Hits,Kudos,Term_novelty,Topic_novelty,Freq_relationship,Category_F_F,...,Fandom_shakespare,Fandom_hetalia,Fandom_attack_on_titan,Fandom_ms_paint_adventures,Fandom_homestuck,Fandom_marvel,Fandom_sailor_moon,Fandom_one_direction,Fandom_sherlock,History
0,0.411820,1.0,3,-0.660467,-0.041314,-0.185058,0.778655,0.599456,0,0,...,0,0,0,0,0,0,0,0,0,5131.0
1,-1.577527,0.0,1,-0.660467,-0.644380,-0.632639,0.713975,0.605999,0,0,...,0,0,0,0,0,0,0,0,0,5584.0


### Logistic

In [12]:
df_all['Intercept'] = 1.0

In [13]:
df_all['kudos_binary'] = df_all['Kudos'].apply(lambda x: 1 if x != 0.0 else 0)

In [14]:
df_all['kudos_binary'].value_counts()

1    701599
Name: kudos_binary, dtype: int64

In [15]:
df_all = df_all.replace([np.inf, -np.inf], np.nan)
df_all = df_all.dropna(how = 'any')

In [16]:
len(df_all)

701599

In [17]:
logit_model = sm.Logit(df_all["kudos_binary"], df_all[['ChapterIndex', 'Chapters',\
        'Term_novelty', 'Topic_novelty', \
        'Freq_relationship', 'Category_F_M',\
       'Category_Gen', 'Category_M_M', 'Category_Multi', 'Category_Other','Category_Unknown', \
       'ArchiveWarnings_underage', 'ArchiveWarnings_death',\
       'ArchiveWarnings_choose_no', 'ArchiveWarnings_no_apply',\
       'ArchiveWarnings_violence',\
       'ArchiveWarnings_noncon', 'author_fic_cnt', 'Rating_G',\
       'Rating_M', 'Rating_N', 'Rating_T',\
       'Fandom_dcu', 'Fandom_doctor_who', 'Fandom_star_wars',
       'Fandom_arthurian', 'Fandom_supernatural', 'Fandom_haikyuu',
       'Fandom_kuroko_no_basuke', 'Fandom_hamilton_miranda',
       'Fandom_dragon_age', 'Fandom_the_walking_dead', 'Fandom_buffy',
       'Fandom_les_miserables', 'Fandom_naruto', 'Fandom_tolkien',
       'Fandom_shakespare', 'Fandom_hetalia', 'Fandom_attack_on_titan',
       'Fandom_ms_paint_adventures', 'Fandom_homestuck', 'Fandom_marvel',
       'Fandom_sailor_moon', 'Fandom_one_direction', 'Fandom_sherlock',
       'History', 'Intercept']]).fit(method='bfgs')

PerfectSeparationError: Perfect separation detected, results not available

In [ ]:
# logit_model.summary()

In [ ]:
pred = logit_model.predict()

In [ ]:
sorted(pred)

In [ ]:
df_all['kudos_nonzero_prob'] = pred

### OLS

In [33]:
df_all_log = df_all.copy()
df_all_log['Kudos'] = np.log(df_all_log['Kudos']+0.001)
df_all_log['Hits'] = np.log(df_all_log['Hits']+0.001)
df_all_log['Comments'] = np.log(df_all_log['Comments']+1)
df_all_log['Bookmarks'] = np.log(df_all_log['Bookmarks']+0.001)
df_all_log = df_all_log.replace([np.inf, -np.inf], np.nan)
df_all_log = df_all_log.dropna(how = 'any')

In [20]:
# s = ''
# for item in df_all.columns.values:
#     s += item + ' + '

In [18]:
model = ols("Kudos ~ Term_novelty + Topic_novelty \
 + ChapterIndex + Chapters + Freq_relationship  + Category_F_M + \
 Category_Gen + Category_M_M + Category_Multi + Category_Other + Category_Unknown \
 + ArchiveWarnings_underage + ArchiveWarnings_death + ArchiveWarnings_choose_no +\
 ArchiveWarnings_no_apply  + ArchiveWarnings_violence +\
 ArchiveWarnings_noncon + author_fic_cnt  + Rating_E + Rating_M + Rating_N + Rating_T\
 + Fandom_dcu + Fandom_doctor_who + Fandom_star_wars + Fandom_arthurian + \
 Fandom_supernatural + Fandom_haikyuu + Fandom_kuroko_no_basuke + Fandom_hamilton_miranda\
 + Fandom_dragon_age + Fandom_the_walking_dead + Fandom_buffy + Fandom_les_miserables \
 + Fandom_naruto + Fandom_tolkien + Fandom_shakespare + Fandom_hetalia + \
 Fandom_attack_on_titan + Fandom_ms_paint_adventures + Fandom_homestuck +\
 Fandom_marvel + Fandom_sailor_moon + Fandom_one_direction + Fandom_sherlock \
 + History + kudos_nonzero_prob", data = df_all).fit()

In [19]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Kudos   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.183
Method:                 Least Squares   F-statistic:                     3488.
Date:                Fri, 13 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:02:36   Log-Likelihood:            -9.2470e+05
No. Observations:              701599   AIC:                         1.849e+06
Df Residuals:                  701553   BIC:                         1.850e+06
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

In [325]:
# for item in ['Term_novelty', 'Topic_novelty']:
#     fig = plt.figure(figsize=(12,8))
#     fig = sm.graphics.plot_regress_exog(model, item, fig=fig)


In [38]:
model = ols("Fandom_dcu ~ Fandom_star_wars + Fandom_arthurian + \
 Fandom_supernatural + Fandom_haikyuu + Fandom_kuroko_no_basuke + Fandom_hamilton_miranda\
 + Fandom_dragon_age + Fandom_the_walking_dead + Fandom_buffy + Fandom_les_miserables \
 + Fandom_naruto + Fandom_tolkien + Fandom_shakespare + Fandom_hetalia + \
 Fandom_attack_on_titan + Fandom_ms_paint_adventures + Fandom_homestuck +\
 Fandom_marvel + Fandom_sailor_moon + Fandom_one_direction + Fandom_sherlock", data = df_all_log).fit()

In [39]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             Fandom_dcu   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                 1.031e+04
Date:                Wed, 11 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:26:22   Log-Likelihood:             1.6231e+05
No. Observations:              701667   AIC:                        -3.246e+05
Df Residuals:                  701645   BIC:                        -3.243e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           